In [17]:
import numpy as np
import os
from pickle import dump
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.resnet50 import preprocess_input

# Define the CNN model to be used for feature extraction
model = ResNet50()

# Set the input shape of the model
input_shape = (224, 224)

# Define the path to the directory containing the images
img_dir_path = "../Flickr8k_Dataset/Flicker8k_Dataset/"

# Get the list of all image filenames in the directory
img_filenames = os.listdir(img_dir_path)

# Define the output directory for the extracted features
output_dir_path = "../Flickr8k_Dataset/features/ResNet50/"

# Create the output directory if it doesn't already exist
if not os.path.exists(output_dir_path):
    os.makedirs(output_dir_path)

# Loop through all image filenames and extract features
for i, img_filename in enumerate(img_filenames):
    # Load the image and resize it to the input shape of the model
    img_path = os.path.join(img_dir_path, img_filename)
    img = load_img(img_path, target_size=input_shape)

    # Convert the image to a numpy array
    img = img_to_array(img)

    # Reshape the image to match the input shape of the model
    img = img.reshape((1,) + img.shape)

    # Preprocess the image for the model
    img = preprocess_input(img)

    # Use the model to extract features from the image
    features = model.predict(img)

    # Save the features to a file with the same name as the image
    out_filename = os.path.splitext(img_filename)[0] + ".npy"
    out_path = os.path.join(output_dir_path, out_filename)
    np.save(out_path, features)

    # Print progress
    print(f"Processed {i+1}/{len(img_filenames)} images: {img_filename}")


1/1 [==============================] - 4s 4s/step
Processed 1/8091 images: 1000268201_693b08cb0e.jpg
1/1 [==============================] - 0s 308ms/step
Processed 2/8091 images: 1001773457_577c3a7d70.jpg
1/1 [==============================] - 0s 310ms/step
Processed 3/8091 images: 1002674143_1b742ab4b8.jpg
1/1 [==============================] - 0s 329ms/step
Processed 4/8091 images: 1003163366_44323f5815.jpg
1/1 [==============================] - 0s 330ms/step
Processed 5/8091 images: 1007129816_e794419615.jpg
1/1 [==============================] - 0s 344ms/step
Processed 6/8091 images: 1007320043_627395c3d8.jpg
1/1 [==============================] - 0s 374ms/step
Processed 7/8091 images: 1009434119_febe49276a.jpg
1/1 [==============================] - 0s 341ms/step
Processed 8/8091 images: 1012212859_01547e3f17.jpg
1/1 [==============================] - 0s 324ms/step
Processed 9/8091 images: 1015118661_980735411b.jpg
1/1 [==============================] - 0s 347ms/step
Processed 10/8

1/1 [==============================] - 0s 405ms/step
Processed 77/8091 images: 109260216_85b0be5378.jpg
1/1 [==============================] - 0s 418ms/step
Processed 78/8091 images: 109260218_fca831f933.jpg
1/1 [==============================] - 0s 454ms/step
Processed 79/8091 images: 1093716555_801aacef79.jpg
1/1 [==============================] - 0s 417ms/step
Processed 80/8091 images: 1093737381_b313cd49ff.jpg
1/1 [==============================] - 0s 364ms/step
Processed 81/8091 images: 1094462889_f9966dafa6.jpg
1/1 [==============================] - 0s 327ms/step
Processed 82/8091 images: 1095476286_87d4f8664e.jpg
1/1 [==============================] - 0s 312ms/step
Processed 83/8091 images: 1095580424_76f0aa8a3e.jpg
1/1 [==============================] - 0s 334ms/step
Processed 84/8091 images: 1095590286_c654f7e5a9.jpg
1/1 [==============================] - 0s 329ms/step
Processed 85/8091 images: 1095980313_3c94799968.jpg
1/1 [==============================] - 0s 361ms/step
Proc

1/1 [==============================] - 0s 329ms/step
Processed 155/8091 images: 1167908324_8caab45e15.jpg
1/1 [==============================] - 0s 309ms/step
Processed 156/8091 images: 1169307342_e7a4685a5c.jpg
1/1 [==============================] - 0s 301ms/step
Processed 157/8091 images: 1174525839_7c1e6cfa86.jpg
1/1 [==============================] - 0s 422ms/step
Processed 158/8091 images: 1174629344_a2e1a2bdbf.jpg
1/1 [==============================] - 0s 408ms/step
Processed 159/8091 images: 1176580356_9810d877bf.jpg
1/1 [==============================] - 0s 377ms/step
Processed 160/8091 images: 1177994172_10d143cb8d.jpg
1/1 [==============================] - 0s 301ms/step
Processed 161/8091 images: 1178705300_c224d9a4f1.jpg
1/1 [==============================] - 0s 308ms/step
Processed 162/8091 images: 118187095_d422383c81.jpg
1/1 [==============================] - 0s 316ms/step
Processed 163/8091 images: 118309463_a532b75be9.jpg
1/1 [==============================] - 0s 307ms/

1/1 [==============================] - 0s 299ms/step
Processed 233/8091 images: 127488876_f2d2a89588.jpg
1/1 [==============================] - 0s 321ms/step
Processed 234/8091 images: 127490019_7c5c08cb11.jpg
1/1 [==============================] - 0s 317ms/step
Processed 235/8091 images: 1277185009_06478dd457.jpg
1/1 [==============================] - 0s 314ms/step
Processed 236/8091 images: 1277743944_f4e8c78403.jpg
1/1 [==============================] - 0s 297ms/step
Processed 237/8091 images: 1280147517_98767ca3b3.jpg
1/1 [==============================] - 0s 317ms/step
Processed 238/8091 images: 1280320287_b2a4b9b7bd.jpg
1/1 [==============================] - 0s 304ms/step
Processed 239/8091 images: 1282392036_5a0328eb86.jpg
1/1 [==============================] - 0s 316ms/step
Processed 240/8091 images: 12830823_87d2654e31.jpg
1/1 [==============================] - 0s 279ms/step
Processed 241/8091 images: 1285067106_2adc307240.jpg
1/1 [==============================] - 0s 323ms/st

1/1 [==============================] - 0s 323ms/step
Processed 310/8091 images: 133905560_9d012b47f3.jpg
1/1 [==============================] - 0s 306ms/step
Processed 311/8091 images: 1339596997_8ac29c1841.jpg
1/1 [==============================] - 0s 296ms/step
Processed 312/8091 images: 1341787777_4f1ebb1793.jpg
1/1 [==============================] - 0s 322ms/step
Processed 313/8091 images: 1342766791_1e72f92455.jpg
1/1 [==============================] - 0s 356ms/step
Processed 314/8091 images: 1342780478_bacc32344d.jpg
1/1 [==============================] - 0s 394ms/step
Processed 315/8091 images: 1343426964_cde3fb54e8.jpg
1/1 [==============================] - 0s 383ms/step
Processed 316/8091 images: 1346051107_9cdc14e070.jpg
1/1 [==============================] - 0s 389ms/step
Processed 317/8091 images: 1346529555_e916816cfe.jpg
1/1 [==============================] - 0s 359ms/step
Processed 318/8091 images: 134724228_30408cd77f.jpg
1/1 [==============================] - 0s 329ms/

...

1/1 [==============================] - 1s 1s/step
Processed 7732/8091 images: 55135290_9bed5c4ca3.jpg
1/1 [==============================] - 1s 1s/step
Processed 7733/8091 images: 551403320_dfdcf9fc3b.jpg
1/1 [==============================] - 1s 1s/step
Processed 7734/8091 images: 551664516_78a5131dc4.jpg
1/1 [==============================] - 1s 1s/step
Processed 7735/8091 images: 554526471_a31f8b74ef.jpg
1/1 [==============================] - 1s 1s/step
Processed 7736/8091 images: 55470226_52ff517151.jpg
1/1 [==============================] - 1s 1s/step
Processed 7737/8091 images: 55473406_1d2271c1f2.jpg
1/1 [==============================] - 1s 1s/step
Processed 7738/8091 images: 554774472_b5d165ff69.jpg
1/1 [==============================] - 1s 1s/step
Processed 7739/8091 images: 556568556_bc5124dc8e.jpg
1/1 [==============================] - 1s 1s/step
Processed 7740/8091 images: 557101732_32bbc47c12.jpg
1/1 [==============================] - 1s 1s/step
Processed 7741/8091 images

1/1 [==============================] - 2s 2s/step
Processed 7812/8091 images: 639120223_7db6bdb61f.jpg
1/1 [==============================] - 1s 1s/step
Processed 7813/8091 images: 639865690_d66d480879.jpg
1/1 [==============================] - 2s 2s/step
Processed 7814/8091 images: 640053014_549d2f23d2.jpg
1/1 [==============================] - 2s 2s/step
Processed 7815/8091 images: 640203018_c0e7175831.jpg
1/1 [==============================] - 2s 2s/step
Processed 7816/8091 images: 640409060_6af18fdd54.jpg
1/1 [==============================] - 1s 1s/step
Processed 7817/8091 images: 640506101_ae1145b6d1.jpg
1/1 [==============================] - 2s 2s/step
Processed 7818/8091 images: 641893280_36fd6e886a.jpg
1/1 [==============================] - 1s 1s/step
Processed 7819/8091 images: 642987597_03b21a1437.jpg
1/1 [==============================] - 2s 2s/step
Processed 7820/8091 images: 649596742_5ba84ce946.jpg
1/1 [==============================] - 1s 1s/step
Processed 7821/8091 ima

1/1 [==============================] - 1s 1s/step
Processed 7892/8091 images: 754852108_72f80d421f.jpg
1/1 [==============================] - 2s 2s/step
Processed 7893/8091 images: 755326139_ee344ece7b.jpg
1/1 [==============================] - 1s 1s/step
Processed 7894/8091 images: 756004341_1a816df714.jpg
1/1 [==============================] - 2s 2s/step
Processed 7895/8091 images: 756521713_5d3da56a54.jpg
1/1 [==============================] - 1s 1s/step
Processed 7896/8091 images: 756909515_a416161656.jpg
1/1 [==============================] - 2s 2s/step
Processed 7897/8091 images: 757046028_ff5999f91b.jpg
1/1 [==============================] - 1s 1s/step
Processed 7898/8091 images: 757133580_ba974ef649.jpg
1/1 [==============================] - 2s 2s/step
Processed 7899/8091 images: 757332692_6866ae545c.jpg
1/1 [==============================] - 2s 2s/step
Processed 7900/8091 images: 758921886_55a351dd67.jpg
1/1 [==============================] - 2s 2s/step
Processed 7901/8091 ima

1/1 [==============================] - 2s 2s/step
Processed 7972/8091 images: 856985136_649c0a3881.jpg
1/1 [==============================] - 2s 2s/step
Processed 7973/8091 images: 857914283_270d7d1c87.jpg
1/1 [==============================] - 1s 1s/step
Processed 7974/8091 images: 859620561_de417cac1e.jpg
1/1 [==============================] - 2s 2s/step
Processed 7975/8091 images: 860928274_744d14f198.jpg
1/1 [==============================] - 2s 2s/step
Processed 7976/8091 images: 861608773_bdafd5c996.jpg
1/1 [==============================] - 2s 2s/step
Processed 7977/8091 images: 861661418_8a37024ace.jpg
1/1 [==============================] - 1s 1s/step
Processed 7978/8091 images: 861795382_5145ad433d.jpg
1/1 [==============================] - 2s 2s/step
Processed 7979/8091 images: 862054277_34b5a6f401.jpg
1/1 [==============================] - 1s 1s/step
Processed 7980/8091 images: 862156271_3eacea90a8.jpg
1/1 [==============================] - 2s 2s/step
Processed 7981/8091 ima

1/1 [==============================] - 1s 1s/step
Processed 8052/8091 images: 95734038_2ab5783da7.jpg
1/1 [==============================] - 2s 2s/step
Processed 8053/8091 images: 957682378_46c3b07bcd.jpg
1/1 [==============================] - 1s 1s/step
Processed 8054/8091 images: 95783195_e1ba3f57ca.jpg
1/1 [==============================] - 2s 2s/step
Processed 8055/8091 images: 958326692_6210150354.jpg
1/1 [==============================] - 2s 2s/step
Processed 8056/8091 images: 961189263_0990f3bcb5.jpg
1/1 [==============================] - 2s 2s/step
Processed 8057/8091 images: 961611340_251081fcb8.jpg
1/1 [==============================] - 2s 2s/step
Processed 8058/8091 images: 963730324_0638534227.jpg
1/1 [==============================] - 2s 2s/step
Processed 8059/8091 images: 96399948_b86c61bfe6.jpg
1/1 [==============================] - 2s 2s/step
Processed 8060/8091 images: 964197865_0133acaeb4.jpg
1/1 [==============================] - 2s 2s/step
Processed 8061/8091 images

In [ ]:
import numpy as np
import os
from pickle import dump
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.inception_v3 import preprocess_input

# Define the CNN model to be used for feature extraction
model = InceptionV3()

# Set the input shape of the model
input_shape = (299, 299)

# Define the path to the directory containing the images
img_dir_path = "Flickr8k_Dataset/Flicker8k_Dataset/"

# Get the list of all image filenames in the directory
img_filenames = os.listdir(img_dir_path)

# Define the output directory for the extracted features
output_dir_path = "Flickr8k_Dataset/features/InceptionV3/"

# Create the output directory if it doesn't already exist
if not os.path.exists(output_dir_path):
    os.makedirs(output_dir_path)

# Loop through all image filenames and extract features
for i, img_filename in enumerate(img_filenames):
    # Load the image and resize it to the input shape of the model
    img_path = os.path.join(img_dir_path, img_filename)
    img = load_img(img_path, target_size=input_shape)

    # Convert the image to a numpy array
    img = img_to_array(img)

    # Reshape the image to match the input shape of the model
    img = img.reshape((1,) + img.shape)

    # Preprocess the image for the model
    img = preprocess_input(img)

    # Use the model to extract features from the image
    features = model.predict(img)

    # Save the features to a file with the same name as the image
    out_filename = os.path.splitext(img_filename)[0] + ".npy"
    out_path = os.path.join(output_dir_path, out_filename)
    np.save(out_path, features)

    # Print progress
    print(f"Processed {i+1}/{len(img_filenames)} images: {img_filename}")


1/1 [==============================] - 12s 12s/step
Processed 1/8091 images: 1000268201_693b08cb0e.jpg
1/1 [==============================] - 1s 1s/step
Processed 2/8091 images: 1001773457_577c3a7d70.jpg
1/1 [==============================] - 1s 1s/step
Processed 3/8091 images: 1002674143_1b742ab4b8.jpg
1/1 [==============================] - 1s 911ms/step
Processed 4/8091 images: 1003163366_44323f5815.jpg
1/1 [==============================] - 1s 897ms/step
Processed 5/8091 images: 1007129816_e794419615.jpg
1/1 [==============================] - 1s 889ms/step
Processed 6/8091 images: 1007320043_627395c3d8.jpg
1/1 [==============================] - 1s 1s/step
Processed 7/8091 images: 1009434119_febe49276a.jpg
1/1 [==============================] - 1s 896ms/step
Processed 8/8091 images: 1012212859_01547e3f17.jpg
1/1 [==============================] - 1s 898ms/step
Processed 9/8091 images: 1015118661_980735411b.jpg
1/1 [==============================] - 1s 905ms/step
Processed 10/8091 ima

1/1 [==============================] - 1s 821ms/step
Processed 78/8091 images: 109260218_fca831f933.jpg
1/1 [==============================] - 1s 836ms/step
Processed 79/8091 images: 1093716555_801aacef79.jpg
1/1 [==============================] - 1s 981ms/step
Processed 80/8091 images: 1093737381_b313cd49ff.jpg
1/1 [==============================] - 1s 971ms/step
Processed 81/8091 images: 1094462889_f9966dafa6.jpg
1/1 [==============================] - 1s 772ms/step
Processed 82/8091 images: 1095476286_87d4f8664e.jpg
1/1 [==============================] - 1s 815ms/step
Processed 83/8091 images: 1095580424_76f0aa8a3e.jpg
1/1 [==============================] - 1s 731ms/step
Processed 84/8091 images: 1095590286_c654f7e5a9.jpg
1/1 [==============================] - 1s 716ms/step
Processed 85/8091 images: 1095980313_3c94799968.jpg
1/1 [==============================] - 1s 812ms/step
Processed 86/8091 images: 1096097967_ac305887b4.jpg
1/1 [==============================] - 1s 800ms/step
Pro

1/1 [==============================] - 4s 4s/step
Processed 157/8091 images: 1174525839_7c1e6cfa86.jpg
1/1 [==============================] - 3s 3s/step
Processed 158/8091 images: 1174629344_a2e1a2bdbf.jpg
1/1 [==============================] - 3s 3s/step
Processed 159/8091 images: 1176580356_9810d877bf.jpg
1/1 [==============================] - 4s 4s/step
Processed 160/8091 images: 1177994172_10d143cb8d.jpg
1/1 [==============================] - 2s 2s/step
Processed 161/8091 images: 1178705300_c224d9a4f1.jpg
1/1 [==============================] - 4s 4s/step
Processed 162/8091 images: 118187095_d422383c81.jpg
1/1 [==============================] - 3s 3s/step
Processed 163/8091 images: 118309463_a532b75be9.jpg
1/1 [==============================] - 2s 2s/step
Processed 164/8091 images: 1184967930_9e29ce380d.jpg
1/1 [==============================] - 3s 3s/step
Processed 165/8091 images: 1187435567_18173c148b.jpg
1/1 [==============================] - 3s 3s/step
Processed 166/8091 images

1/1 [==============================] - 1s 769ms/step
Processed 236/8091 images: 1277743944_f4e8c78403.jpg
1/1 [==============================] - 1s 843ms/step
Processed 237/8091 images: 1280147517_98767ca3b3.jpg
1/1 [==============================] - 1s 737ms/step
Processed 238/8091 images: 1280320287_b2a4b9b7bd.jpg
1/1 [==============================] - 1s 822ms/step
Processed 239/8091 images: 1282392036_5a0328eb86.jpg
1/1 [==============================] - 1s 747ms/step
Processed 240/8091 images: 12830823_87d2654e31.jpg
1/1 [==============================] - 1s 1s/step
Processed 241/8091 images: 1285067106_2adc307240.jpg
1/1 [==============================] - 1s 800ms/step
Processed 242/8091 images: 1285874746_486731a954.jpg
1/1 [==============================] - 1s 825ms/step
Processed 243/8091 images: 1286408831_05282582ed.jpg
1/1 [==============================] - 1s 758ms/step
Processed 244/8091 images: 1287064529_aa4e4f3c31.jpg
1/1 [==============================] - 1s 819ms/ste

Processed 313/8091 images: 1342766791_1e72f92455.jpg
1/1 [==============================] - 1s 753ms/step
Processed 314/8091 images: 1342780478_bacc32344d.jpg
1/1 [==============================] - 1s 815ms/step
Processed 315/8091 images: 1343426964_cde3fb54e8.jpg
1/1 [==============================] - 1s 920ms/step
Processed 316/8091 images: 1346051107_9cdc14e070.jpg
1/1 [==============================] - 1s 875ms/step
Processed 317/8091 images: 1346529555_e916816cfe.jpg
1/1 [==============================] - 1s 1s/step
Processed 318/8091 images: 134724228_30408cd77f.jpg
1/1 [==============================] - 1s 788ms/step
Processed 319/8091 images: 1347519824_e402241e4f.jpg
1/1 [==============================] - 1s 819ms/step
Processed 320/8091 images: 1348113612_5bfc5f429e.jpg
1/1 [==============================] - 1s 742ms/step
Processed 321/8091 images: 1348304997_afe60a61df.jpg
1/1 [==============================] - 1s 830ms/step
Processed 322/8091 images: 1348891916_ebd4413033.j

...

1/1 [==============================] - 1s 1s/step
Processed 5754/8091 images: 3458577912_67db47209d.jpg
1/1 [==============================] - 1s 1s/step
Processed 5755/8091 images: 3458625738_297857369c.jpg
1/1 [==============================] - 1s 1s/step
Processed 5756/8091 images: 3459156091_c1879ebe28.jpg
1/1 [==============================] - 1s 1s/step
Processed 5757/8091 images: 3459362347_c412ef9901.jpg
1/1 [==============================] - 1s 1s/step
Processed 5758/8091 images: 3459419203_cd7c68ce4d.jpg
1/1 [==============================] - 1s 1s/step
Processed 5759/8091 images: 3459492423_c881f12c9f.jpg
1/1 [==============================] - 1s 1s/step
Processed 5760/8091 images: 3459570613_3932816d3f.jpg
1/1 [==============================] - 1s 1s/step
Processed 5761/8091 images: 3459858555_c3f0087a72.jpg
1/1 [==============================] - 1s 1s/step
Processed 5762/8091 images: 3459871361_92d1ecda36.jpg
1/1 [==============================] - 1s 1s/step
Processed 5763

1/1 [==============================] - 3s 3s/step
Processed 5833/8091 images: 3474999131_788cbf253f.jpg
1/1 [==============================] - 2s 2s/step
Processed 5834/8091 images: 3475005101_6f6e437459.jpg
1/1 [==============================] - 2s 2s/step
Processed 5835/8091 images: 3475092236_cf45d383c7.jpg
1/1 [==============================] - 2s 2s/step
Processed 5836/8091 images: 3475111806_f0d2927707.jpg
1/1 [==============================] - 2s 2s/step
Processed 5837/8091 images: 347543966_b2053ae78c.jpg
1/1 [==============================] - 2s 2s/step
Processed 5838/8091 images: 3475552729_a3abd81ee6.jpg
1/1 [==============================] - 2s 2s/step
Processed 5839/8091 images: 3475581086_a533567561.jpg
1/1 [==============================] - 2s 2s/step
Processed 5840/8091 images: 3476237185_9389c536a3.jpg
1/1 [==============================] - 2s 2s/step
Processed 5841/8091 images: 3476381830_3751dd9339.jpg
1/1 [==============================] - 2s 2s/step
Processed 5842/

1/1 [==============================] - 1s 1s/step
Processed 5912/8091 images: 3490517179_76dbd690de.jpg
1/1 [==============================] - 1s 1s/step
Processed 5913/8091 images: 3490528249_6aae9b867b.jpg
1/1 [==============================] - 1s 1s/step
Processed 5914/8091 images: 3490597800_8f94f7d353.jpg
1/1 [==============================] - 1s 1s/step
Processed 5915/8091 images: 3490736665_38710f4b91.jpg
1/1 [==============================] - 1s 1s/step
Processed 5916/8091 images: 3490867290_13bcd3a7f0.jpg
1/1 [==============================] - 1s 1s/step
Processed 5917/8091 images: 3490874218_babb404b39.jpg
1/1 [==============================] - 1s 1s/step
Processed 5918/8091 images: 3491013009_572cf2c18a.jpg
1/1 [==============================] - 1s 1s/step
Processed 5919/8091 images: 3491607076_922ec561d9.jpg
1/1 [==============================] - 1s 1s/step
Processed 5920/8091 images: 3492180255_0bd48a18f8.jpg
1/1 [==============================] - 1s 1s/step
Processed 5921

1/1 [==============================] - 2s 2s/step
Processed 5991/8091 images: 3506468593_7e41a6d9f1.jpg
1/1 [==============================] - 2s 2s/step
Processed 5992/8091 images: 3506560025_8d0f4f9ac4.jpg
1/1 [==============================] - 2s 2s/step
Processed 5993/8091 images: 3506607642_40037b3fbf.jpg
1/1 [==============================] - 2s 2s/step
Processed 5994/8091 images: 3506869953_802f463178.jpg
1/1 [==============================] - 2s 2s/step
Processed 5995/8091 images: 3507076266_8b17993fbb.jpg
1/1 [==============================] - 2s 2s/step
Processed 5996/8091 images: 3507670136_2e5f94accf.jpg
1/1 [==============================] - 2s 2s/step
Processed 5997/8091 images: 3508051251_82422717b3.jpg
1/1 [==============================] - 2s 2s/step
Processed 5998/8091 images: 3508522093_51f6d77f45.jpg
1/1 [==============================] - 2s 2s/step
Processed 5999/8091 images: 3508637029_89f3bdd3a2.jpg
1/1 [==============================] - 2s 2s/step
Processed 6000

1/1 [==============================] - 2s 2s/step
Processed 6070/8091 images: 3523559027_a65619a34b.jpg
1/1 [==============================] - 1s 1s/step
Processed 6071/8091 images: 3523819210_99782628fc.jpg
1/1 [==============================] - 2s 2s/step
Processed 6072/8091 images: 352382023_7605223d1c.jpg
1/1 [==============================] - 2s 2s/step
Processed 6073/8091 images: 3523874798_9ba2fa46e3.jpg
1/1 [==============================] - 2s 2s/step
Processed 6074/8091 images: 3523920786_0eb63993fd.jpg
1/1 [==============================] - 1s 1s/step
Processed 6075/8091 images: 3523950181_414978964e.jpg
1/1 [==============================] - 2s 2s/step
Processed 6076/8091 images: 3523972229_d44e9ff6d7.jpg
1/1 [==============================] - 3s 3s/step
Processed 6077/8091 images: 3524436870_7670df68e8.jpg
1/1 [==============================] - 3s 3s/step
Processed 6078/8091 images: 3524519277_bd0c3e7382.jpg
1/1 [==============================] - 1s 1s/step
Processed 6079/

In [ ]:
import numpy as np
import os
from pickle import dump
from tensorflow.keras.applications.efficientnet import EfficientNetB7
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.efficientnet import preprocess_input

# Define the CNN model to be used for feature extraction
model = EfficientNetB7()

# Set the input shape of the model
input_shape = (600, 600)

# Define the path to the directory containing the images
img_dir_path = "Flickr8k_Dataset/Flicker8k_Dataset/"

# Get the list of all image filenames in the directory
img_filenames = os.listdir(img_dir_path)

# Define the output directory for the extracted features
output_dir_path = "Flickr8k_Dataset/features/EfficientNetB7/"

# Create the output directory if it doesn't already exist
if not os.path.exists(output_dir_path):
    os.makedirs(output_dir_path)

# Loop through all image filenames and extract features
for i, img_filename in enumerate(img_filenames):
    # Load the image and resize it to the input shape of the model
    img_path = os.path.join(img_dir_path, img_filename)
    img = load_img(img_path, target_size=input_shape)

    # Convert the image to a numpy array
    img = img_to_array(img)

    # Reshape the image to match the input shape of the model
    img = img.reshape((1,) + img.shape)

    # Preprocess the image for the model
    img = preprocess_input(img)

    # Use the model to extract features from the image
    features = model.predict(img)

    # Save the features to a file with the same name as the image
    out_filename = os.path.splitext(img_filename)[0] + ".npy"
    out_path = os.path.join(output_dir_path, out_filename)
    np.save(out_path, features)

    # Print progress
    print(f"Processed {i+1}/{len(img_filenames)} images: {img_filename}")


In [ ]:
import os
import numpy as np
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Define the CNN model to be used for feature extraction
model = InceptionResNetV2(weights='imagenet', include_top=False)

# Set the input shape of the model
input_shape = (299, 299)

# Define the path to the directory containing the images
img_dir_path = "Flickr8k_Dataset/Flicker8k_Dataset/"

# Get the list of all image filenames in the directory
img_filenames = os.listdir(img_dir_path)

# Define the output directory for the extracted features
output_dir_path = "Flickr8k_Dataset/features/InceptionResNetV2/"

# Create the output directory if it doesn't already exist
if not os.path.exists(output_dir_path):
    os.makedirs(output_dir_path)

# Loop through all image filenames and extract features
for i, img_filename in enumerate(img_filenames):
    # Load the image and resize it to the input shape of the model
    img_path = os.path.join(img_dir_path, img_filename)
    img = load_img(img_path, target_size=input_shape)

    # Convert the image to a numpy array
    img = img_to_array(img)

    # Reshape the image to match the input shape of the model
    img = img.reshape((1,) + img.shape)

    # Preprocess the image for the model
    img = preprocess_input(img)

    # Use the model to extract features from the image
    features = model.predict(img)

    # Flatten the features and save to a file with the same name as the image
    features = features.flatten()
    out_filename = os.path.splitext(img_filename)[0] + ".npy"
    out_path = os.path.join(output_dir_path, out_filename)
    np.save(out_path, features)

    # Print progress
    print(f"Processed {i+1}/{len(img_filenames)} images: {img_filename}")
